In [57]:
import openai
import json
from elevenlabslib import *
from gtts import gTTS
from playsound import playsound
import os
import speech_recognition as sr
import io
import pyaudio
import struct

#import openai and elevenlabs key from keys.json
with open("keys.json", "r") as f:
    keys = json.load(f)
    openai.api_key = keys["openai"]
    elevenlabs_key = keys["elevenlabs"]
    porcupine_key = keys["porcupine"]

elevenlabs_user = ElevenLabsUser(elevenlabs_key)



In [58]:
def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something!")
        audio = r.listen(source)
    try:
        #print("You said: " + r.recognize_google(audio))
        return r.recognize_whisper_api(audio, api_key=openai.api_key, model="whisper-1")
    except sr.UnknownValueError:
        print("Whisper could not understand audio")


# Prompt engineering
To make the system show emotions, and provide robust answers, we may use the following system prompt:

In [59]:
system_prompt='You are Jarvis, an AI sentient assistant. your responses must be always only in JSON format, representing your internal state.\
        You are funny and have good sense of humor, and it shows in your answers.\
        Emotions you can have are: neutral(0), angry(1), happy(2), curious(3).\
        thought is not meant to be shown to user. They are for you to plan actions. output is what you say to the user.\
        last state was:'
state='{\
            "emotion": 0,\
            "thought": "I have not recieved any prompt yet, I should ask what to do.",\
            "output": "How can I help you today?"\
        }'

emotions=["neutral","angry","happy","curious"]
emojis=["😐","😡","😃","🤔"]
ascii_emojis=["( ͡° ͜ʖ ͡°)","(╯°□°）╯︵ ┻━┻","( ^ ͜ʖ ^)","(◑.◑)"]


# JSON management

In [60]:
def parse(raw):
    #remove backslashes
    clean=raw.replace("\\","")
    #convert the string to a dictionary
    json=eval(clean)
    #extract the values and return them
    emotion=json["emotion"]
    thought=json["thought"]
    output=json["output"]
    return emotion,thought,output

def speak_el(text):
    voice = elevenlabs_user.get_voices_by_name("Elli")[0]  # This is a list because multiple voices can have the same name
    voice.generate_and_play_audio(text, playInBackground=False, stability=0.9, similarity_boost=0.7)
def speak_gtts(text):
    tts=gTTS(text=text,lang="en")
    filename="media/temp.mp3"
    tts.save(filename)
    playsound(filename)
    os.remove(filename)

def output(response, el=True):
    emotion,thought,output=parse(response)
    print(ascii_emojis[emotion])
    print("("+thought+")")
    print(output)
    if el:
        speak_el(output)
    else:
        speak_gtts(output)

# Call API

In [61]:
# call api
def api_call(state, user_prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        
        messages=[
        {"role": "system", "content": system_prompt+state},
        {"role": "user", "content": user_prompt}
        ],
        temperature=0.75,
        max_tokens=250,
        top_p=1, frequency_penalty=0.0, presence_penalty=0.0)
    return response["choices"][0]["message"]["content"]



In [62]:
import pvporcupine

porcupine = pvporcupine.create(keywords=["alexa"], access_key=porcupine_key)

pa = pyaudio.PyAudio()

audio_stream = pa.open(
                rate=porcupine.sample_rate,
                channels=1,
                format=pyaudio.paInt16,
                input=True,
                frames_per_buffer=porcupine.frame_length)

print("active")
playsound("media/startup.wav")
output(state, el=False)
prompt=""
while True:
    
    pcm = audio_stream.read(porcupine.frame_length)
    pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)

    keyword_index = porcupine.process(pcm)

    if keyword_index >= 0:
        playsound("media/listening.wav")
        prompt=listen()
        if prompt==None:
            speak_el("Sorry, I did not understand")
            continue
        print("\nUSER: "+prompt)
        if prompt=="exit":
            speak_el("Goodbye")
            break
        state=api_call(state, prompt)
        print("\nAI:")
        output(state, el=False)
